## Import Packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, label_ranking_average_precision_score, label_ranking_loss, roc_auc_score
from sklearn.preprocessing import label_binarize
import ast

## Load the dataset

In [ ]:
# Load data
df = pd.read_excel("./dataset/train_data.xlsx", engine='openpyxl')
test_df = pd.read_excel("./dataset/test_data.xlsx", engine='openpyxl')

## Data Pre-processing

In [ ]:
climate_features = [
    'tavg_2020_mean', 'prec_2020_mean',
    'tavg_2021_mean', 'prec_2021_mean',
    'tavg_2022_mean', 'prec_2022_mean',
    'tavg_2023_mean', 'prec_2023_mean',
    'tavg_2024_mean', 'prec_2024_mean'
]

numeric_features = ['latitude', 'longitude', 'elevation']
categorical_features = ['Land_Cover_Type']

In [ ]:
# Transform the training data
X_climate_train = df[climate_features].fillna(0)
scaler_climate = StandardScaler()
X_scaled_train = scaler_climate.fit_transform(X_climate_train)

pca = PCA(n_components=2, random_state=42)
climate_pca_train = pca.fit_transform(X_scaled_train)

df['climate_PC1'] = climate_pca_train[:, 0]
df['climate_PC2'] = climate_pca_train[:, 1]

print("Train PCA - Explained variance:", pca.explained_variance_ratio_)
print("Train PCA - Cumulative variance:", np.cumsum(pca.explained_variance_ratio_))

In [ ]:
# Transform test data using previously fitted scaler and PCA
X_climate_test = test_df[climate_features].fillna(0)
X_scaled_test = scaler_climate.transform(X_climate_test)
climate_pca_test = pca.transform(X_scaled_test)

test_df['climate_PC1'] = climate_pca_test[:, 0]
test_df['climate_PC2'] = climate_pca_test[:, 1]

df.drop(columns=climate_features, inplace=True, errors='ignore')
test_df.drop(columns=climate_features, inplace=True, errors='ignore')

In [ ]:
#Preprocess the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features + ['climate_PC1', 'climate_PC2']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='drop'
)

# Fit on training data
X_processed = preprocessor.fit_transform(df)
print("Train processed shape:", X_processed.shape)

# Transform test data
X_test_processed = preprocessor.transform(test_df)
print("Test processed shape:", X_test_processed.shape)


In [ ]:
#Target labels
y = df['train_ids']
y_test = test_df['species']

# Training
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Test (multilabel)
y_test_lists = [ast.literal_eval(str(species_str)) if isinstance(species_str, str)
                else species_str for species_str in y_test]
y_test_enc = [le.transform(species_list) for species_list in y_test_lists]

# Convert the test to binary matrix
mlb = MultiLabelBinarizer(classes=range(len(le.classes_)))
y_test_multilabel = mlb.fit_transform(y_test_enc)

print(f"Test multilabel shape: {y_test_multilabel.shape}")
print(f"Avg species per test sample: {y_test_multilabel.sum(axis=1).mean():.2f}")

In [ ]:
y_encoded.shape

In [ ]:
y_test_multilabel.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split( X_processed, y_encoded, test_size=0.1, random_state=42, stratify=y_encoded )

print(f"Train: {X_train.shape}, Val: {X_val.shape}")

## Calculating weights (Upweighting)

In [ ]:
def calculate_class_weights(y_train):
    counts = pd.Series(y_train).value_counts()
    class_weights = {cls: 1.0/np.sqrt(count) for cls, count in counts.items()}
    return class_weights

class_weights = calculate_class_weights(y_train)

## Model Training and Evaluation


In [ ]:
import time
import psutil
import os

def train_and_evaluate(model, model_name, X_train, y_train, class_weights=None):

    print(f"Training: {model_name}")

    process = psutil.Process(os.getpid())

    # Memory before training
    mem_before = process.memory_info().rss / 1024 / 1024  # MB

    # Train
    start = time.time()
    if class_weights is not None and hasattr(model, 'class_weight'):
        model.set_params(class_weight=class_weights)
    model.fit(X_train, y_train)
    train_time = time.time() - start

    # Memory after training
    mem_after = process.memory_info().rss / 1024 / 1024  # MB
    mem_used = mem_after - mem_before

    print(f"Training completed in {train_time:.2f} seconds")
    print(f"Memory used: {mem_used:.2f} MB (Total: {mem_after:.2f} MB)")

    return model, train_time, mem_used

## Hyperparameter search

In [ ]:
import matplotlib.pyplot as plt

def plot_metric(df, train_col, val_col, title, y_label):
    plt.figure(figsize=(10, 6))

    colors = ['blue', 'green','orange']
    split_vals = sorted(df['min_samples_split'].unique())

    for i, split_val in enumerate(split_vals):
        subset = df[df['min_samples_split'] == split_val]
        curr_color = colors[i % len(colors)]

        plt.plot(subset['max_depth'], subset[val_col],
                 marker='o', label=f'Val (split={split_val})',
                 color=curr_color, linestyle='-')

    plt.title(title, fontsize=14)
    plt.xlabel('Max Depth', fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

def plot_xgb_metric(df, train_col, val_col, title, y_label):
    plt.figure(figsize=(10, 6))

    colors = ['blue', 'orange']
    est_vals = sorted(df['n_estimators'].unique())

    for i, n_est in enumerate(est_vals):
        subset = df[df['n_estimators'] == n_est]
        curr_color = colors[i % len(colors)]

        plt.plot(subset['max_depth'], subset[val_col],
                 marker='o', label=f'Val (trees={n_est})',
                 color=curr_color, linestyle='-')

        plt.plot(subset['max_depth'], subset[train_col],
                 label=f'Train (trees={n_est})',
                 color=curr_color, linestyle='--', alpha=0.6)

    plt.title(title, fontsize=14)
    plt.xlabel('Max Depth', fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.xticks(df['max_depth'].unique())
    plt.grid(True, alpha=0.3)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
depth_range = [5, 10, 15, 20, 25]
split_range = [5, 10, 15, 20, 25]

results_list = []

for min_split in split_range:
    for depth in depth_range:

        print(f"Training DT: Depth={depth}, Split={min_split}..")

        dt = DecisionTreeClassifier(
            max_depth=depth,
            min_samples_split=min_split,
            random_state=42
        )

        dt.fit(X_train, y_train)

        y_train_probs = dt.predict_proba(X_train)
        train_metrics = evaluate_model(y_train, y_train_probs, is_multilabel=False, model_name="DT", split_name="Train")

        y_val_probs = dt.predict_proba(X_val)
        val_metrics = evaluate_model(y_val, y_val_probs, is_multilabel=False, model_name="DT", split_name="Val")

        results_list.append({
            'max_depth': depth,
            'min_samples_split': min_split,

            'Train_AUC': train_metrics['AUC-ROC'],
            'Val_AUC': val_metrics['AUC-ROC'],

            'Train_Top5': train_metrics['Top-5'],
            'Val_Top5': val_metrics['Top-5'],

            'Train_Top10': train_metrics['Top-10'],
            'Val_Top10': val_metrics['Top-10']
        })

df_plot = pd.DataFrame(results_list)

In [ ]:
plot_metric(df_plot, 'Train_AUC', 'Val_AUC',
            'DT Hyperparameter Search: AUC-ROC', 'AUC-ROC Score')

plot_metric(df_plot, 'Train_Top5', 'Val_Top5',
            'DT Hyperparameter Search: Top-5 Accuracy', 'Top-5 Score')

plot_metric(df_plot, 'Train_Top10', 'Val_Top10',
            'DT Hyperparameter Search: Top-10 Accuracy', 'Top-10 Score')

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
estimator_range = [50, 100, 200]
depth_range = [5, 8, 10]
split_range = [5, 10]

rf_results_list = []

for min_split in split_range:
    for depth in depth_range:
      for estimate in estimator_range:
          print(f"Training RF: Depth={depth}, Split={min_split}, Estimator={estimate}...")

          rf = RandomForestClassifier(
              n_estimators=estimate,
              max_depth=depth,
              min_samples_split=min_split,
              class_weight=class_weights,
              random_state=42,
              n_jobs=-1
          )

          rf.fit(X_train, y_train)

          y_train_probs = rf.predict_proba(X_train)
          train_metrics = evaluate_model(y_train, y_train_probs, is_multilabel=False, model_name="RF", split_name="Train")

          y_val_probs = rf.predict_proba(X_val)
          val_metrics = evaluate_model(y_val, y_val_probs, is_multilabel=False, model_name="RF", split_name="Val")

          rf_results_list.append({
              'max_depth': depth,
              'min_samples_split': min_split,

              'Train_AUC': train_metrics['AUC-ROC'],
              'Val_AUC': val_metrics['AUC-ROC'],

              'Train_Top5': train_metrics['Top-5'],
              'Val_Top5': val_metrics['Top-5'],

              'Train_Top10': train_metrics['Top-10'],
              'Val_Top10': val_metrics['Top-10']
          })

df_rf_plot = pd.DataFrame(rf_results_list)

In [ ]:
plot_metric(df_rf_plot, 'Train_AUC', 'Val_AUC',
            'RF Hyperparameter Search: AUC-ROC', 'AUC-ROC Score')

plot_metric(df_rf_plot, 'Train_Top5', 'Val_Top5',
            'RF Hyperparameter Search: Top-5 Accuracy', 'Top-5 Score')

plot_metric(df_rf_plot, 'Train_Top10', 'Val_Top10',
            'RF Hyperparameter Search: Top-10 Accuracy', 'Top-10 Score')

XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier
estimators_range = [50, 100]
depth_range = [5, 6]

xgb_results_list = []


for n_est in estimators_range:
    for depth in depth_range:
        print(f"Training XGB: Trees={n_est}, Depth={depth}...")

        xgb = XGBClassifier(
            n_estimators=n_est,
            max_depth=depth,
            learning_rate=0.1,
            random_state=42,
            n_jobs=-1,
            eval_metric='mlogloss'
        )
        xgb, xgb_time, xgb_mem = train_and_evaluate(xgb, "XGBoost", X_train, y_train)

        y_train_probs = xgb.predict_proba(X_train)
        train_metrics = evaluate_model(y_train, y_train_probs, is_multilabel=False, model_name="XGB", split_name="Train")

        y_val_probs = xgb.predict_proba(X_val)
        val_metrics = evaluate_model(y_val, y_val_probs, is_multilabel=False, model_name="XGB", split_name="Val")

        xgb_results_list.append({
            'n_estimators': n_est,
            'max_depth': depth,

            'Train_AUC': train_metrics['AUC-ROC'],
            'Val_AUC': val_metrics['AUC-ROC'],

            'Train_Top5': train_metrics['Top-5'],
            'Val_Top5': val_metrics['Top-5'],

            'Train_Top10': train_metrics['Top-10'],
            'Val_Top10': val_metrics['Top-10']
        })

df_xgb_plot = pd.DataFrame(xgb_results_list)

In [ ]:
plot_xgb_metric(df_xgb_plot, 'Train_AUC', 'Val_AUC',
            'XGBoost Hyperparameter Search: AUC-ROC', 'AUC-ROC Score')

plot_xgb_metric(df_xgb_plot, 'Train_Top5', 'Val_Top5',
            'XGBoost Hyperparameter Search: Top-5 Accuracy', 'Top-5 Score')

plot_xgb_metric(df_xgb_plot, 'Train_Top10', 'Val_Top10',
            'XGBoost Hyperparameter Search: Top-10 Accuracy', 'Top-10 Score')

Extra Tree Classifier

In [ ]:
import matplotlib.pyplot as plt

max_depth_list = [5, 10, 20, 30, 50]
min_split_list = [5, 10, 20]

hp_results = []


for min_split in min_split_list:
    for max_depth in max_depth_list:
        print(f"\nTesting max_depth={max_depth}, min_samples_split={min_split}")

        fold_val_top5 = []
        fold_val_top10 = []
        fold_train_top5 = []
        fold_train_top10 = []
        fold_auc = []

        for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train), 1):

            X_train_split, X_val_split = X_train[train_idx], X_train[val_idx]
            y_train_split, y_val_split = y_train[train_idx], y_train[val_idx]

            class_counts = Counter(y_train_split)
            class_weights = {label: 1 / np.sqrt(count) for label, count in class_counts.items()}
            sample_weights = np.array([class_weights[label] for label in y_train_split])

            model = ExtraTreesClassifier(
                n_estimators=100,
                max_depth=max_depth,
                min_samples_split=min_split,
                min_samples_leaf=5,
                class_weight=class_weights,
                random_state=42,
                n_jobs=-1
            )

            model.fit(X_train_split, y_train_split, sample_weight=sample_weights)

            y_val_pred = model.predict_proba(X_val_split)
            y_train_pred = model.predict_proba(X_train_split)

            val_top5 = top_k_accuracy_score(y_val_split, y_val_pred, k=5)
            val_top10 = top_k_accuracy_score(y_val_split, y_val_pred, k=10)

            train_top5 = top_k_accuracy_score(y_train_split, y_train_pred, k=5)
            train_top10 = top_k_accuracy_score(y_train_split, y_train_pred, k=10)

            auc_roc = roc_auc_score(
                label_binarize(y_val_split, classes=np.unique(y_train)),
                y_val_pred,
                average='weighted',
                multi_class='ovr'
            )

            fold_val_top5.append(val_top5)
            fold_val_top10.append(val_top10)
            fold_train_top5.append(train_top5)
            fold_train_top10.append(train_top10)
            fold_auc.append(auc_roc)

        hp_results.append({
            "max_depth": max_depth,
            "min_split": min_split,
            "train_top5_mean": np.mean(fold_train_top5),
            "train_top10_mean": np.mean(fold_train_top10),
            "val_top5_mean": np.mean(fold_val_top5),
            "val_top10_mean": np.mean(fold_val_top10),
            "auc_mean": np.mean(fold_auc),
        })

hp_df = pd.DataFrame(hp_results)
print("Hyperparameter Search Completed!")
hp_df

In [ ]:
plt.figure(figsize=(12,7))

jitter = {5: -1.0, 10: 0, 20: 1.0}

for min_split in min_split_list:
    subdf = hp_df[hp_df["min_split"] == min_split]

    plt.plot(
        subdf["max_depth"] + jitter[min_split],
        subdf["val_top10_mean"],
        marker="o",
        linestyle="--",
        label=f"Val Top-10 (min_split={min_split})"
    )

    plt.plot(
        subdf["max_depth"] + jitter[min_split],
        subdf["train_top10_mean"],
        marker="s",
        linestyle="-",
        label=f"Train Top-10 (min_split={min_split})"
    )

plt.title("Extra Trees Hyperparameter Search (Top-10 Accuracy: Train vs Validation)")
plt.xlabel("Max Depth")
plt.ylabel("Top-10 Accuracy")
plt.legend(ncol=2)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12,7))

for min_split in min_split_list:
    subdf = hp_df[hp_df["min_split"] == min_split]

    # Validation Top-5
    plt.plot(
        subdf["max_depth"] + jitter[min_split],
        subdf["val_top5_mean"],
        marker="o",
        linestyle="--",
        label=f"Val Top-5 (min_split={min_split})"
    )

    # Training Top-5
    plt.plot(
        subdf["max_depth"] + jitter[min_split],
        subdf["train_top5_mean"],
        marker="s",
        linestyle="-",
        label=f"Train Top-5 (min_split={min_split})"
    )

plt.title("Extra Trees Hyperparameter Search (Top-5 Accuracy: Train vs Validation)")
plt.xlabel("Max Depth")
plt.ylabel("Top-5 Accuracy")
plt.legend(ncol=2)
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score, top_k_accuracy_score
import matplotlib.pyplot as plt
import psutil, os, time


def top_k_acc(model, X, y, k):
    """Compute top-k accuracy."""
    probs = model.predict_proba(X)
    return top_k_accuracy_score(y, probs, k=k, labels=np.arange(probs.shape[1]))

def train_and_evaluate(model, X_train, y_train, X_val, y_val):
    """Returns metrics for train & val."""

    process = psutil.Process(os.getpid())
    mem_before = process.memory_info().rss / 1024 / 1024

    start = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start

    mem_after = process.memory_info().rss / 1024 / 1024
    mem_used = mem_after - mem_before

    train_probs = model.predict_proba(X_train)
    val_probs   = model.predict_proba(X_val)

    results = {
        "train_top5": top_k_accuracy_score(y_train, train_probs, k=5),
        "val_top5":   top_k_accuracy_score(y_val,   val_probs,   k=5),

        "train_top10": top_k_accuracy_score(y_train, train_probs, k=10),
        "val_top10":   top_k_accuracy_score(y_val,   val_probs,   k=10),

        "train_auc": roc_auc_score(y_train, train_probs, multi_class='ovr'),
        "val_auc":   roc_auc_score(y_val,   val_probs,   multi_class='ovr'),

        "train_time": train_time,
        "mem_used_MB": mem_used
    }
    return results

max_depth_list = [5, 8, 10]
split_list     = [2, 5, 10]

results = []

for depth in max_depth_list:
    for split in split_list:
        print(f"Running ExtraTrees with depth={depth}, min_samples_split={split}")

        model = ExtraTreesClassifier(
            n_estimators=100,
            max_depth=depth,
            min_samples_split=split,
            class_weight=class_weights,
            n_jobs=-1,
            random_state=42
        )

        metrics = train_and_evaluate(model, X_train, y_train, X_val, y_val)
        metrics["max_depth"] = depth
        metrics["min_samples_split"] = split

        results.append(metrics)

df = pd.DataFrame(results)
print(df)

In [ ]:
def plot_metric(df, metric_train, metric_val, title):
    plt.figure(figsize=(10,6))

    for split in split_list:
        subset = df[df["min_samples_split"] == split]

        plt.plot(subset["max_depth"], subset[metric_train],
                 marker='o', label=f"Train - split={split}")

        plt.plot(subset["max_depth"], subset[metric_val],
                 marker='o', linestyle='--', label=f"Val - split={split}")

    plt.xlabel("max_depth")
    plt.ylabel(metric_train.replace("_", " ").title())
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

plot_metric(df, "train_top5", "val_top5",
            "ExtraTrees Hyperparameter Search — Top-5 Accuracy")

plot_metric(df, "train_top10", "val_top10",
            "ExtraTrees Hyperparameter Search — Top-10 Accuracy")

plot_metric(df, "train_auc", "val_auc",
            "ExtraTrees Hyperparameter Search — AUC-ROC")

## Stratified Cross Validation with best parameters for each model

Decision Tree

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import top_k_accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
from collections import Counter
import time

# Initialize
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Storage for results
cv_results = {
    'fold': [],
    'top5_acc': [],
    'top10_acc': [],
    'auc_roc': [],
    'species_coverage': [],
    'training_time': []
}

# Cross-validation loop
for fold, (train_idx, val_idx) in enumerate(skf.split(X_processed, y_encoded), 1):
    print(f"Training Fold {fold}/{n_splits}...")

    # Split data
    X_train_split, X_val_split = X_processed[train_idx], X_processed[val_idx]
    y_train_split, y_val_split = y_encoded[train_idx], y_encoded[val_idx]

    # Calculate class weights
    class_counts = Counter(y_train_split)
    class_weights = {label: 1/np.sqrt(count) for label, count in class_counts.items()}
    sample_weights = np.array([class_weights[label] for label in y_train_split])

    # Train Decision Tree
    start_time = time.time()

    dt = DecisionTreeClassifier(
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        class_weight=class_weights,
        random_state=42
    )

    dt.fit(X_train_split, y_train_split, sample_weight=sample_weights)
    training_time = time.time() - start_time

    # Predictions
    y_pred_proba = dt.predict_proba(X_val_split)

    # Calculate metrics
    top5 = top_k_accuracy_score(y_val_split, y_pred_proba, k=5)
    top10 = top_k_accuracy_score(y_val_split, y_pred_proba, k=10)

    # AUC-ROC
    y_val_binarized = label_binarize(y_val_split, classes=np.unique(y_train))
    if y_val_binarized.shape[1] == y_pred_proba.shape[1]:
        auc_roc = roc_auc_score(y_val_binarized, y_pred_proba, average='weighted', multi_class='ovr')
    else:
        common_classes = np.unique(y_val_split)
        y_val_binarized_subset = label_binarize(y_val_split, classes=common_classes)
        y_pred_proba_subset = y_pred_proba[:, common_classes]
        auc_roc = roc_auc_score(y_val_binarized_subset, y_pred_proba_subset, average='weighted', multi_class='ovr')

    # Species Coverage
    species_coverage = np.mean(np.sum(y_pred_proba > 0.00, axis=1))

    # Store results
    cv_results['fold'].append(fold)
    cv_results['top5_acc'].append(top5)
    cv_results['top10_acc'].append(top10)
    cv_results['auc_roc'].append(auc_roc)
    cv_results['species_coverage'].append(species_coverage)
    cv_results['training_time'].append(training_time)

    print(f"  Top-5: {top5:.4f}, Top-10: {top10:.4f}, AUC-ROC: {auc_roc:.4f}, Coverage: {species_coverage:.2f}")

cv_df_dt = pd.DataFrame(cv_results)

# Summary statistics
summary_dt = {
    'Model': 'Decision Tree',
    'Top-5 Mean': cv_df_dt['top5_acc'].mean(),
    'Top-5 Std': cv_df_dt['top5_acc'].std(),
    'Top-10 Mean': cv_df_dt['top10_acc'].mean(),
    'Top-10 Std': cv_df_dt['top10_acc'].std(),
    'AUC-ROC Mean': cv_df_dt['auc_roc'].mean(),
    'AUC-ROC Std': cv_df_dt['auc_roc'].std(),
    'Species Coverage Mean': cv_df_dt['species_coverage'].mean(),
    'Species Coverage Std': cv_df_dt['species_coverage'].std(),
    'Training Time Mean': cv_df_dt['training_time'].mean(),
    'Training Time Std': cv_df_dt['training_time'].std()
}

print()
print("Decision Tree Metrics:")
for key, value in summary_dt.items():
    if 'Mean' in key or 'Std' in key:
        print(f"{key:25s}: {value:.4f}")
    else:
        print(f"{key:25s}: {value}")

Random Forest

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import top_k_accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Storage for results
cv_results = {
    'fold': [],
    'top5_acc': [],
    'top10_acc': [],
    'auc_roc': [],
    'avg_species_coverage': [],
    'training_time': []
}

# Cross-validation loop
for fold, (train_idx, val_idx) in enumerate(skf.split(X_processed, y_encoded), 1):
    print(f"Training Fold {fold}/{n_splits}")

    # Split data
    X_train_split, X_val_split = X_processed[train_idx], X_processed[val_idx]
    y_train_split, y_val_split = y_encoded[train_idx], y_encoded[val_idx]

    # Calculate class weights
    from collections import Counter
    class_counts = Counter(y_train_split)
    class_weights = {label: 1/np.sqrt(count) for label, count in class_counts.items()}
    sample_weights = np.array([class_weights[label] for label in y_train_split])

    # Train model
    import time
    start_time = time.time()

    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=5,
        min_samples_split=10,
        class_weight=class_weights,
        random_state=42,
        n_jobs=-1
    )

    rf.fit(X_train_split, y_train_split, sample_weight=sample_weights)
    training_time = time.time() - start_time

    # Predictions
    y_pred_proba = rf.predict_proba(X_val_split)

    # Metrics
    top5 = top_k_accuracy_score(y_val_split, y_pred_proba, k=5)
    top10 = top_k_accuracy_score(y_val_split, y_pred_proba, k=10)

    # AUC-ROC
    y_val_binarized = label_binarize(y_val_split, classes=np.unique(y_train))
    if y_val_binarized.shape[1] == y_pred_proba.shape[1]:
        auc_roc = roc_auc_score(y_val_binarized, y_pred_proba, average='weighted', multi_class='ovr')
    else:
        common_classes = np.unique(y_val_split)
        y_val_binarized_subset = label_binarize(y_val_split, classes=common_classes)
        y_pred_proba_subset = y_pred_proba[:, common_classes]
        auc_roc = roc_auc_score(y_val_binarized_subset, y_pred_proba_subset, average='weighted', multi_class='ovr')

    species_coverage = np.sum(y_pred_proba > 0.00, axis=1)  # Count species with >0% probability per sample
    avg_species_coverage = np.mean(species_coverage)

    # Store results
    cv_results['fold'].append(fold)
    cv_results['top5_acc'].append(top5)
    cv_results['top10_acc'].append(top10)
    cv_results['auc_roc'].append(auc_roc)
    cv_results['avg_species_coverage'].append(avg_species_coverage)
    cv_results['training_time'].append(training_time)

    print(f"Top-5: {top5:.4f}, Top-10: {top10:.4f}, AUC-ROC: {auc_roc:.4f}, Avg Species Coverage: {avg_species_coverage:.2f}")

# Convert to DataFrame
cv_df = pd.DataFrame(cv_results)

print(cv_df)

for metric in ['top5_acc', 'top10_acc', 'auc_roc', 'avg_species_coverage']:
    print(f"{metric:20s}: {cv_df[metric].mean():.4f} ± {cv_df[metric].std():.4f}")


XGBoost Classifier

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import top_k_accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
from collections import Counter
import time

# Initialize
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Storage for results
cv_results = {
    'fold': [],
    'top5_acc': [],
    'top10_acc': [],
    'auc_roc': [],
    'species_coverage': [],
    'training_time': []
}

for fold, (train_idx, val_idx) in enumerate(skf.split(X_processed, y_encoded), 1):
    print(f"Training Fold {fold}/{n_splits}...")

    # Split data
    X_train_split, X_val_split = X_processed[train_idx], X_processed[val_idx]
    y_train_split, y_val_split = y_encoded[train_idx], y_encoded[val_idx]

    # Calculate class weights
    class_counts = Counter(y_train_split)
    class_weights = {label: 1/np.sqrt(count) for label, count in class_counts.items()}
    sample_weights = np.array([class_weights[label] for label in y_train_split])

    # Train XGBoost
    start_time = time.time()

    xgb = XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        class_weights=class_weights,
        objective='multi:softprob',  # For soft probabilities, model calibration
        random_state=42,
        n_jobs=-1,
        eval_metric='mlogloss'
        )

    xgb.fit(X_train_split, y_train_split, sample_weight=sample_weights)
    training_time = time.time() - start_time

    # Predictions
    y_pred_proba = xgb.predict_proba(X_val_split)

    # Calculate metrics
    top5 = top_k_accuracy_score(y_val_split, y_pred_proba, k=5)
    top10 = top_k_accuracy_score(y_val_split, y_pred_proba, k=10)

    # AUC-ROC
    y_val_binarized = label_binarize(y_val_split, classes=np.unique(y_train))
    if y_val_binarized.shape[1] == y_pred_proba.shape[1]:
        auc_roc = roc_auc_score(y_val_binarized, y_pred_proba, average='weighted', multi_class='ovr')
    else:
        common_classes = np.unique(y_val_split)
        y_val_binarized_subset = label_binarize(y_val_split, classes=common_classes)
        y_pred_proba_subset = y_pred_proba[:, common_classes]
        auc_roc = roc_auc_score(y_val_binarized_subset, y_pred_proba_subset, average='weighted', multi_class='ovr')

    # Species Coverage
    species_coverage = np.mean(np.sum(y_pred_proba > 0.00, axis=1))

    # Store results
    cv_results['fold'].append(fold)
    cv_results['top5_acc'].append(top5)
    cv_results['top10_acc'].append(top10)
    cv_results['auc_roc'].append(auc_roc)
    cv_results['species_coverage'].append(species_coverage)
    cv_results['training_time'].append(training_time)

    print(f"  Top-5: {top5:.4f}, Top-10: {top10:.4f}, AUC-ROC: {auc_roc:.4f}, Coverage: {species_coverage:.2f}")

# Convert to DataFrame
cv_df_xgb = pd.DataFrame(cv_results)

# Summary statistics
summary_xgb = {
    'Model': 'XGBoost',
    'Top-5 Mean': cv_df_xgb['top5_acc'].mean(),
    'Top-5 Std': cv_df_xgb['top5_acc'].std(),
    'Top-10 Mean': cv_df_xgb['top10_acc'].mean(),
    'Top-10 Std': cv_df_xgb['top10_acc'].std(),
    'AUC-ROC Mean': cv_df_xgb['auc_roc'].mean(),
    'AUC-ROC Std': cv_df_xgb['auc_roc'].std(),
    'Species Coverage Mean': cv_df_xgb['species_coverage'].mean(),
    'Species Coverage Std': cv_df_xgb['species_coverage'].std(),
    'Training Time Mean': cv_df_xgb['training_time'].mean(),
    'Training Time Std': cv_df_xgb['training_time'].std()
}

print("XGBoost - Summary:")
for key, value in summary_xgb.items():
    if 'Mean' in key or 'Std' in key:
        print(f"{key:25s}: {value:.4f}")
    else:
        print(f"{key:25s}: {value}")

Extra Tree Classifier

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import top_k_accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
import pandas as pd
from collections import Counter
import time

# Initialize
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Storage for results
cv_results = {
    'fold': [],
    'top5_acc': [],
    'top10_acc': [],
    'auc_roc': [],
    'species_coverage': [],
    'training_time': []
}

# Cross-validation loop
for fold, (train_idx, val_idx) in enumerate(skf.split(X_processed, y_encoded), 1):
    print(f"Training Fold {fold}/{n_splits}...")

    # Split data
    X_train_split, X_val_split = X_processed[train_idx], X_processed[val_idx]
    y_train_split, y_val_split = y_encoded[train_idx], y_encoded[val_idx]

    # Calculate class weights
    class_counts = Counter(y_train_split)
    class_weights = {label: 1/np.sqrt(count) for label, count in class_counts.items()}
    sample_weights = np.array([class_weights[label] for label in y_train_split])

    # Train Extra Trees
    start_time = time.time()

    et = ExtraTreesClassifier(
        n_estimators=100,
        max_depth=5,
        min_samples_split=10,
        min_samples_leaf=5,
        class_weight=class_weights,
        random_state=42,
        n_jobs=-1
    )

    et.fit(X_train_split, y_train_split, sample_weight=sample_weights)
    training_time = time.time() - start_time

    # Predictions
    y_pred_proba = et.predict_proba(X_val_split)

    # Calculate metrics
    top5 = top_k_accuracy_score(y_val_split, y_pred_proba, k=5)
    top10 = top_k_accuracy_score(y_val_split, y_pred_proba, k=10)

    # AUC-ROC
    y_val_binarized = label_binarize(y_val_split, classes=np.unique(y_train))
    if y_val_binarized.shape[1] == y_pred_proba.shape[1]:
        auc_roc = roc_auc_score(y_val_binarized, y_pred_proba, average='weighted', multi_class='ovr')
    else:
        common_classes = np.unique(y_val_split)
        y_val_binarized_subset = label_binarize(y_val_split, classes=common_classes)
        y_pred_proba_subset = y_pred_proba[:, common_classes]
        auc_roc = roc_auc_score(y_val_binarized_subset, y_pred_proba_subset, average='weighted', multi_class='ovr')

    # Species Coverage
    species_coverage = np.mean(np.sum(y_pred_proba > 0.01, axis=1))

    # Store results
    cv_results['fold'].append(fold)
    cv_results['top5_acc'].append(top5)
    cv_results['top10_acc'].append(top10)
    cv_results['auc_roc'].append(auc_roc)
    cv_results['species_coverage'].append(species_coverage)
    cv_results['training_time'].append(training_time)

    print(f"  Top-5: {top5:.4f}, Top-10: {top10:.4f}, AUC-ROC: {auc_roc:.4f}, Coverage: {species_coverage:.2f}")

# Convert to DataFrame
cv_df_et = pd.DataFrame(cv_results)

# Summary statistics
summary_et = {
    'Model': 'Extra Trees',
    'Top-5 Mean': cv_df_et['top5_acc'].mean(),
    'Top-5 Std': cv_df_et['top5_acc'].std(),
    'Top-10 Mean': cv_df_et['top10_acc'].mean(),
    'Top-10 Std': cv_df_et['top10_acc'].std(),
    'AUC-ROC Mean': cv_df_et['auc_roc'].mean(),
    'AUC-ROC Std': cv_df_et['auc_roc'].std(),
    'Species Coverage Mean': cv_df_et['species_coverage'].mean(),
    'Species Coverage Std': cv_df_et['species_coverage'].std(),
    'Training Time Mean': cv_df_et['training_time'].mean(),
    'Training Time Std': cv_df_et['training_time'].std()
}

print()
print("Extra Trees - Summary:")
for key, value in summary_et.items():
    if 'Mean' in key or 'Std' in key:
        print(f"{key:25s}: {value:.4f}")
    else:
        print(f"{key:25s}: {value}")

Light Gradient Boost Model

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import top_k_accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
from lightgbm import LGBMClassifier
import numpy as np
import pandas as pd
from collections import Counter
import time

# Initialize
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Storage for results
cv_results = {
    'fold': [],
    'top5_acc': [],
    'top10_acc': [],
    'auc_roc': [],
    'species_coverage': [],
    'training_time': []
}

# Cross-validation loop
for fold, (train_idx, val_idx) in enumerate(skf.split(X_processed, y_encoded), 1):
    print(f"Training Fold {fold}/{n_splits}...")

    # Split data
    X_train_split, X_val_split = X_processed[train_idx], X_processed[val_idx]
    y_train_split, y_val_split = y_encoded[train_idx], y_encoded[val_idx]

    # Calculate class weights
    class_counts = Counter(y_train_split)
    class_weights = {label: 1/np.sqrt(count) for label, count in class_counts.items()}
    sample_weights = np.array([class_weights[label] for label in y_train_split])

    # Train LightGBM
    start_time = time.time()

    lgbm = LGBMClassifier(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        class_weight=class_weights,
        objective='multiclass',
        num_class=500,  # Number of classes
        random_state=42,
        n_jobs=-1,
        verbosity=-1
    )

    lgbm.fit(X_train_split, y_train_split, sample_weight=sample_weights)
    training_time = time.time() - start_time

    # Predictions
    y_pred_proba = lgbm.predict_proba(X_val_split)

    # Calculate metrics
    top5 = top_k_accuracy_score(y_val_split, y_pred_proba, k=5)
    top10 = top_k_accuracy_score(y_val_split, y_pred_proba, k=10)

    # AUC-ROC
    y_val_binarized = label_binarize(y_val_split, classes=np.unique(y_train))
    if y_val_binarized.shape[1] == y_pred_proba.shape[1]:
        auc_roc = roc_auc_score(y_val_binarized, y_pred_proba, average='weighted', multi_class='ovr')
    else:
        common_classes = np.unique(y_val_split)
        y_val_binarized_subset = label_binarize(y_val_split, classes=common_classes)
        y_pred_proba_subset = y_pred_proba[:, common_classes]
        auc_roc = roc_auc_score(y_val_binarized_subset, y_pred_proba_subset, average='weighted', multi_class='ovr')

    # Species Coverage
    species_coverage = np.mean(np.sum(y_pred_proba > 0.01, axis=1))

    # Store results
    cv_results['fold'].append(fold)
    cv_results['top5_acc'].append(top5)
    cv_results['top10_acc'].append(top10)
    cv_results['auc_roc'].append(auc_roc)
    cv_results['species_coverage'].append(species_coverage)
    cv_results['training_time'].append(training_time)

    print(f"  Top-5: {top5:.4f}, Top-10: {top10:.4f}, AUC-ROC: {auc_roc:.4f}, Coverage: {species_coverage:.2f}")

# Convert to DataFrame
cv_df_lgbm = pd.DataFrame(cv_results)

# Summary statistics
summary_lgbm = {
    'Model': 'LightGBM',
    'Top-5 Mean': cv_df_lgbm['top5_acc'].mean(),
    'Top-5 Std': cv_df_lgbm['top5_acc'].std(),
    'Top-10 Mean': cv_df_lgbm['top10_acc'].mean(),
    'Top-10 Std': cv_df_lgbm['top10_acc'].std(),
    'AUC-ROC Mean': cv_df_lgbm['auc_roc'].mean(),
    'AUC-ROC Std': cv_df_lgbm['auc_roc'].std(),
    'Species Coverage Mean': cv_df_lgbm['species_coverage'].mean(),
    'Species Coverage Std': cv_df_lgbm['species_coverage'].std(),
    'Training Time Mean': cv_df_lgbm['training_time'].mean(),
    'Training Time Std': cv_df_lgbm['training_time'].std()
}

print()
print("LightGBM - Summary:")
for key, value in summary_lgbm.items():
    if 'Mean' in key or 'Std' in key:
        print(f"{key:25s}: {value:.4f}")
    else:
        print(f"{key:25s}: {value}")

## Evaluating Test set

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

# Evaluation function
def evaluate_test_set(model, X_test, y_test_multilabel, model_name):
    y_pred_proba = model.predict_proba(X_test)

    # Top-5 and Top-10
    top5_hits = []
    top10_hits = []

    for i in range(len(y_test_multilabel)):
        true_species = set(np.where(y_test_multilabel[i] == 1)[0])
        top5_pred = set(np.argsort(y_pred_proba[i])[-5:][::-1])
        top10_pred = set(np.argsort(y_pred_proba[i])[-10:][::-1])

        top5_hits.append(len(true_species & top5_pred) > 0)
        top10_hits.append(len(true_species & top10_pred) > 0)

    # AUC-ROC
    try:
        auc_roc = roc_auc_score(y_test_multilabel, y_pred_proba, average='weighted', multi_class='ovr')
    except:
        auc_roc = np.nan

    # Species Coverage
    species_coverage = np.mean(np.sum(y_pred_proba > 0.01, axis=1))

    return {
        'Model': model_name,
        'Top-5': np.mean(top5_hits),
        'Top-10': np.mean(top10_hits),
        'AUC-ROC': auc_roc,
        'Species Coverage': species_coverage
    }

# Evaluate your models
test_results = []

test_results.append(evaluate_test_set(dt, X_test_processed, y_test_multilabel, 'Decision Tree'))
test_results.append(evaluate_test_set(rf, X_test_processed, y_test_multilabel, 'Random Forest'))
test_results.append(evaluate_test_set(xgb, X_test_processed, y_test_multilabel, 'XGBoost'))

# Create DataFrame
test_df = pd.DataFrame(test_results)
print(test_df)

In [ ]:
test_results = evaluate_test_set(et, X_test_processed, y_test_multilabel, 'Extra Tree')
test_results

In [ ]:
def evaluate_test_set(model, X_test, y_test_multilabel, model_name):
    y_pred_proba = model.predict_proba(X_test)

    # Top-5 and Top-10
    top5_hits = []
    top10_hits = []

    for i in range(len(y_test_multilabel)):
        true_species = set(np.where(y_test_multilabel[i] == 1)[0])
        top5_pred = set(np.argsort(y_pred_proba[i])[-5:][::-1])
        top10_pred = set(np.argsort(y_pred_proba[i])[-10:][::-1])

        top5_hits.append(len(true_species & top5_pred) > 0)
        top10_hits.append(len(true_species & top10_pred) > 0)

    # AUC-ROC
    try:
        auc_roc = roc_auc_score(y_test_multilabel, y_pred_proba, average='weighted', multi_class='ovr')
    except:
        auc_roc = np.nan

    # Species Coverage
    species_coverage = np.mean(np.sum(y_pred_proba > 0.01, axis=1))

    return {
        'Model': model_name,
        'Top-5': np.mean(top5_hits),
        'Top-10': np.mean(top10_hits),
        'AUC-ROC': auc_roc,
        'Species Coverage': species_coverage
    }

In [ ]:
test_results = evaluate_test_set(lgbm, X_test_processed, y_test_multilabel, 'Light GBM')
test_results

#One vs Rest on the best model

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Convert single-label train to multilabel format
y_train_lists = [[label] for label in y_train]
y_val_lists = [[label] for label in y_val]

mlb_train = MultiLabelBinarizer(classes=range(len(le.classes_)))
y_train_multilabel = mlb_train.fit_transform(y_train_lists)
y_val_multilabel = mlb_train.transform(y_val_lists)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

ovr_base = RandomForestClassifier(
    n_estimators=50,
    max_depth=5,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
ovr = MultiOutputClassifier(ovr_base, n_jobs=-1)

In [ ]:
print(f"Training: One-vs-Rest (500 classifiers)")
start = time.time()
ovr.fit(X_train, y_train_multilabel)
ovr_train_time = time.time() - start
print(f"Training completed in {ovr_train_time:.2f} seconds")

In [ ]:
y_val_probs_ovr = np.zeros((len(X_val), len(le.classes_)))
for i, estimator in enumerate(ovr.estimators_):
    y_val_probs_ovr[:, i] = estimator.predict_proba(X_val)[:, 1]

val_results_ovr = evaluate_model(y_val_multilabel, y_val_probs_ovr, is_multilabel=True,
                                  model_name="One-vs-Rest", split_name="Validation")

In [ ]:
y_test_probs_ovr = np.zeros((len(X_test_processed), len(le.classes_)))
for i, estimator in enumerate(ovr.estimators_):
    y_test_probs_ovr[:, i] = estimator.predict_proba(X_test_processed)[:, 1]


test_results_ovr = evaluate_model(y_test_multilabel, y_test_probs_ovr, is_multilabel=True,
                                   model_name="One-vs-Rest", split_name="Test")

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

def evaluate_ovr(model, X, y_multilabel, split_name="Val"):

    print(f"\nEvaluating OvR on {split_name} Set...")

    y_pred_proba_list = model.predict_proba(X)
    y_pred_proba = np.array([y_pred_proba_list[i][:, 1] for i in range(len(y_pred_proba_list))]).T

    # Top-5 and Top-10
    top5_hits = []
    top10_hits = []

    for i in range(len(y_multilabel)):
        true_species = set(np.where(y_multilabel[i] == 1)[0])
        top5_pred = set(np.argsort(y_pred_proba[i])[-5:][::-1])
        top10_pred = set(np.argsort(y_pred_proba[i])[-10:][::-1])

        top5_hits.append(len(true_species & top5_pred) > 0)
        top10_hits.append(len(true_species & top10_pred) > 0)

    top5_acc = np.mean(top5_hits)
    top10_acc = np.mean(top10_hits)

    # AUC-ROC
    try:
        auc_roc = roc_auc_score(y_multilabel, y_pred_proba, average='weighted', multi_class='ovr')
    except:
        auc_roc = np.nan

    # Species Coverage
    species_coverage = np.mean(np.sum(y_pred_proba > 0.01, axis=1))

    print(f"  Top-5 Accuracy:      {top5_acc:.4f}")
    print(f"  Top-10 Accuracy:     {top10_acc:.4f}")
    print(f"  AUC-ROC:             {auc_roc:.4f}")
    print(f"  Species Coverage:    {species_coverage:.2f}")

    return {
        'Model': 'One-vs-Rest',
        'Split': split_name,
        'Top-5': top5_acc,
        'Top-10': top10_acc,
        'AUC-ROC': auc_roc,
        'Species Coverage': species_coverage
    }

# Validation
ovr_val_results = evaluate_ovr(ovr, X_val, y_val_multilabel, "Validation")

# Test
ovr_test_results = evaluate_ovr(ovr, X_test_processed, y_test_multilabel, "Test")

ovr_results_df = pd.DataFrame([ovr_val_results, ovr_test_results])

print()
print("ONE-VS-REST RESULTS:")
print(ovr_results_df)

#Classifier Chain on the best model

In [ ]:
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier

cc_base = RandomForestClassifier(
    n_estimators=50,
    max_depth=5,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
cc = ClassifierChain(cc_base, order='random', random_state=42)

In [ ]:
print(f"Training: Classifier Chain")
start = time.time()
cc.fit(X_train, y_train_multilabel)
cc_train_time = time.time() - start
print(f"Training completed in {cc_train_time:.2f} seconds")

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

def evaluate_classifier_chain(model, X, y_multilabel, split_name="Val"):

    print(f"\nEvaluating Classifier Chain on {split_name} Set...")

    y_pred_proba = model.predict_proba(X)

    # If it returns binary predictions instead of probabilities, use predict
    if y_pred_proba.max() <= 1 and y_pred_proba.min() >= 0:
        pass
    else:
        y_pred_proba = model.predict(X).astype(float)

    # Top-5 and Top-10
    top5_hits = []
    top10_hits = []

    for i in range(len(y_multilabel)):
        true_species = set(np.where(y_multilabel[i] == 1)[0])
        top5_pred = set(np.argsort(y_pred_proba[i])[-5:][::-1])
        top10_pred = set(np.argsort(y_pred_proba[i])[-10:][::-1])

        top5_hits.append(len(true_species & top5_pred) > 0)
        top10_hits.append(len(true_species & top10_pred) > 0)

    top5_acc = np.mean(top5_hits)
    top10_acc = np.mean(top10_hits)

    # AUC-ROC
    try:
        auc_roc = roc_auc_score(y_multilabel, y_pred_proba, average='weighted', multi_class='ovr')
    except:
        auc_roc = np.nan

    # Species Coverage
    species_coverage = np.mean(np.sum(y_pred_proba > 0.01, axis=1))

    print(f"  Top-5 Accuracy:      {top5_acc:.4f}")
    print(f"  Top-10 Accuracy:     {top10_acc:.4f}")
    print(f"  AUC-ROC:             {auc_roc:.4f}")
    print(f"  Species Coverage:    {species_coverage:.2f}")

    return {
        'Model': 'Classifier Chain',
        'Split': split_name,
        'Top-5': top5_acc,
        'Top-10': top10_acc,
        'AUC-ROC': auc_roc,
        'Species Coverage': species_coverage
    }

# Validation
cc_val_results = evaluate_classifier_chain(cc, X_val, y_val_multilabel, "Validation")

# Test
cc_test_results = evaluate_classifier_chain(cc, X_test_processed, y_test_multilabel, "Test")

cc_results_df = pd.DataFrame([cc_val_results, cc_test_results])

print()
print("CLASSIFIER CHAIN RESULTS:")
print(cc_results_df)

In [ ]:
y_val_probs_cc = cc.predict_proba(X_val)

val_results_cc = evaluate_model(y_val_multilabel, y_val_probs_cc, is_multilabel=True,
                                 model_name="Classifier Chain", split_name="Validation")

In [ ]:
y_test_probs_cc = cc.predict_proba(X_test_processed)
test_results_cc = evaluate_model(y_test_multilabel, y_test_probs_cc, is_multilabel=True,
                                  model_name="Classifier Chain", split_name="Test")

## Error and Performance Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    min_samples_split=10,
    class_weight=class_weights,
    random_state=42,
    n_jobs=-1
)
rf, rf_time, rf_mem = train_and_evaluate(rf, "Random Forest", X_train, y_train)

In [ ]:
print("Generating predictions from Random Forest on test set")
y_test_pred_proba = rf.predict_proba(X_test_processed)
y_test_pred = rf.predict(X_test_processed)

print(f"Test predictions shape: {y_test_pred_proba.shape}")

All Species Performance Analysis

In [ ]:
species_freq = Counter(y_train)

species_performance = []

for species_id in range(500):
    # Get samples where this species is present in test set
    species_mask = y_test_multilabel[:, species_id] == 1
    n_samples = species_mask.sum()

    if n_samples == 0:
        continue

    # Check if species appears in top-5 predictions
    top5_preds = np.argsort(y_test_pred_proba, axis=1)[:, -5:]
    species_in_top5 = np.any(top5_preds == species_id, axis=1)

    top5_recall = np.mean(species_in_top5[species_mask])

    # Average probability assigned to this species
    avg_prob = np.mean(y_test_pred_proba[species_mask, species_id])

    species_performance.append({
        'species_id': species_id,
        'train_frequency': species_freq.get(species_id, 0),
        'test_samples': int(n_samples),
        'top5_recall': top5_recall,
        'avg_probability': avg_prob
    })

species_perf_df = pd.DataFrame(species_performance)

# Categorize by frequency
species_perf_df['freq_category'] = pd.cut(
    species_perf_df['train_frequency'],
    bins=[0, 250, 750, 3000],
    labels=['Rare (<250)', 'Medium (250-750)', 'Frequent (>750)']
)

print(f"\nTotal species evaluated: {len(species_perf_df)}")
print(f"\nPerformance by frequency category:")
print(species_perf_df.groupby('freq_category')['top5_recall'].agg(['mean', 'std', 'count']))

In [ ]:
#Category wise plot

plt.figure(figsize=(10, 7))
species_perf_df.boxplot(column='top5_recall', by='freq_category',
                         patch_artist=True, figsize=(10, 7))
plt.suptitle('')
plt.title('Top-5 Recall by Species Frequency Category', fontsize=15, fontweight='bold')
plt.xlabel('Species Frequency Category', fontsize=13, fontweight='bold')
plt.ylabel('Top-5 Recall', fontsize=13, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('performance_by_category.png', dpi=300, bbox_inches='tight')
plt.show()

Most Confused Pairs

In [ ]:
# Find top-1 predictions for each test sample
top1_preds = np.argmax(y_test_pred_proba, axis=1)

# Build confusion matrix (only the top confusions)
confusion_pairs = []

for i in range(len(y_test_multilabel)):
    true_species = np.where(y_test_multilabel[i] == 1)[0]
    pred_species = top1_preds[i]

    # If prediction is wrong
    if pred_species not in true_species:
        for true_sp in true_species:
            confusion_pairs.append({
                'true_species': true_sp,
                'predicted_species': pred_species,
                'true_freq': species_freq.get(true_sp, 0),
                'pred_freq': species_freq.get(pred_species, 0)
            })

confusion_df = pd.DataFrame(confusion_pairs)

if len(confusion_df) > 0:
    confusion_counts = confusion_df.groupby(['true_species', 'predicted_species']).size()
    top_confusions = confusion_counts.nlargest(15)

    print(f"\nTop 15 Most Confused Species Pairs:")
    for (true_sp, pred_sp), count in top_confusions.items():
        print(f"True: {true_sp:3d} Predicted as: {pred_sp:3d} | Count: {count:3d}")

    # Create subset confusion matrix for top confused species
    top_confused_species = set()
    for (true_sp, pred_sp), _ in top_confusions.head(20).items():
        top_confused_species.add(true_sp)
        top_confused_species.add(pred_sp)

    top_confused_species = sorted(list(top_confused_species))[:20]

    # Build confusion matrix subset
    conf_matrix = np.zeros((len(top_confused_species), len(top_confused_species)))

    for (true_sp, pred_sp), count in confusion_counts.items():
        if true_sp in top_confused_species and pred_sp in top_confused_species:
            true_idx = top_confused_species.index(true_sp)
            pred_idx = top_confused_species.index(pred_sp)
            conf_matrix[true_idx, pred_idx] = count

    plt.figure(figsize=(12, 10))
    sns.heatmap(conf_matrix, annot=False, cmap='YlOrRd',
                xticklabels=top_confused_species,
                yticklabels=top_confused_species,
                cbar_kws={'label': 'Confusion Count'})
    plt.xlabel('Predicted Species', fontsize=13, fontweight='bold')
    plt.ylabel('True Species', fontsize=13, fontweight='bold')
    plt.title('Confusion Matrix - Top 20 Most Confused Species', fontsize=15, fontweight='bold')
    plt.tight_layout()
    plt.savefig('confusion_heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()

Feature Importance

In [ ]:
feature_importance = rf.feature_importances_

feature_names = (numeric_features + ['climate_PC1', 'climate_PC2'] +
                 list(preprocessor.named_transformers_['cat'].get_feature_names_out()))

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print(f"\nTop 20 Most Important Features:")
print(importance_df.head(20))

In [ ]:
plt.figure(figsize=(12, 8))
top_features = importance_df.head(20)
plt.barh(range(len(top_features)), top_features['importance'],
         color='blue', edgecolor='black', linewidth=1)
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance', fontsize=13, fontweight='bold')
plt.ylabel('Feature', fontsize=13, fontweight='bold')
plt.title('Top 20 Most Important Features - Random Forest', fontsize=15, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

Other Analysis

In [ ]:
max_probs = np.max(y_test_pred_proba, axis=1)

species_coverage = np.sum(y_test_pred_proba > 0.01, axis=1)

correct_predictions = []
for i in range(len(y_test_multilabel)):
    true_species = set(np.where(y_test_multilabel[i] == 1)[0])
    top5_pred = set(np.argsort(y_test_pred_proba[i])[-5:][::-1])
    correct_predictions.append(len(true_species & top5_pred) > 0)

correct_predictions = np.array(correct_predictions)

print(f"\nConfidence Statistics:")
print(f"Mean max probability: {max_probs.mean():.4f}")
print(f"Mean species coverage: {species_coverage.mean():.2f}")
print(f"Correct predictions: {correct_predictions.sum()} / {len(correct_predictions)}")


In [ ]:
worst_species = species_perf_df.nsmallest(20, 'top5_recall')

print("\nTop 20 Worst Performing Species:")
print(worst_species[['species_id', 'train_frequency', 'test_samples', 'top5_recall', 'avg_probability']])

In [ ]:
confidence_bins = pd.cut(max_probs, bins=[0, 0.1, 0.2, 0.3, 0.5, 1.0],
                         labels=['0-0.1', '0.1-0.2', '0.2-0.3', '0.3-0.5', '0.5-1.0'])

accuracy_by_confidence = []
for bin_label in ['0-0.1', '0.1-0.2', '0.2-0.3', '0.3-0.5', '0.5-1.0']:
    mask = confidence_bins == bin_label
    if mask.sum() > 0:
        acc = correct_predictions[mask].mean()
        count = mask.sum()
        accuracy_by_confidence.append({
            'confidence_bin': bin_label,
            'accuracy': acc,
            'count': count
        })

acc_conf_df = pd.DataFrame(accuracy_by_confidence)
print("\nAccuracy by Confidence Level:")
print(acc_conf_df)